Install dependencies

In [ ]:
!pip3 install torch torchvision torchaudio
!pip3 install matplotlib
!pip3 install ultralytics roboflow

Start the training

In [3]:
import torch
from ultralytics import YOLO
from pathlib import Path
import os
import json
import yaml

In [4]:

# Check for GPU availability

device = "cuda:0" if torch.cuda.is_available() else "cpu"

print(f"Using device: {device} ({'GPU' if device != 'cpu' else 'CPU'})")


Using device: cuda:0 (GPU)


In [ ]:

CONFIG = {
    'model': 'yolo11m.pt',  # Choose model size: n, s, m, l, x
    'data': 'datasets/dataset_v1/data.yaml',  
    'epochs': 100,
    'batch': 6 if device != 'cpu' else 4,  # Adjust batch size for CPU
    'imgsz': 640,
    'patience': 20,
    'device': device, 
}

save_dir = Path('runs/detect')
save_dir.mkdir(parents=True, exist_ok=True)

this_path = os.getcwd()

os.environ['ULTRALYTICS_CONFIG_DIR'] = this_path
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

data_file = f'{this_path}/datasets/dataset_v1/data.yaml'
with open(data_file, 'r') as f:
    data = yaml.safe_load(f)
    

data['train'] = f'{this_path}/datasets/dataset_v1/train/images'
data['val'] = f'{this_path}/datasets/dataset_v1/valid/images'
data['test'] = f'{this_path}/datasets/dataset_v1/test/images'

with open(data_file, 'w') as f:
    yaml.safe_dump(data, f)


In [11]:

model = YOLO(CONFIG['model'])


In [12]:

results = model.train(
    data=CONFIG['data'],
    epochs=CONFIG['epochs'],
    batch=CONFIG['batch'],
    imgsz=CONFIG['imgsz'],
    patience=CONFIG['patience'],
    device=CONFIG['device'],
    
    optimizer='Adam',
    lr0=0.001,
    lrf=0.1,
    momentum=0.9,
    weight_decay=0.0005,
    warmup_epochs=3,
    warmup_momentum=0.8,
    
    # Augmentations
    augment=True,
    hsv_h=0.015,  # Image HSV-Hue augmentation
    hsv_s=0.7,   # Image HSV-Saturation augmentation
    hsv_v=0.4,   # Image HSV-Value augmentation
    degrees=45,  # Image rotation (+/- deg)
    translate=0.1,  # Image translation (+/- fraction)
    scale=0.5,   # Image scale (+/- gain)
    shear=0.0,   # Image shear (+/- deg)
    perspective=0.0,  # Image perspective
    flipud=0.5,  # Image flip up-down
    fliplr=0.5,  # Image flip left-right
    mosaic=1.0,  # Image mosaic
    mixup=0.0,   # Image mixup
    
    save_dir=save_dir
)


Ultralytics 8.3.34 🚀 Python-3.12.7 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650, 3721MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=datasets/dataset_v1/data.yaml, epochs=100, time=None, patience=20, batch=6, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf

train: Scanning /var/home/larizmen/DATA/REPOS/ai-apps/dev/safety-hat/datasets/dataset_v1/train/labels.cache... 1191 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1191/1191 [00:00<?, ?it/s]
val: Scanning /var/home/larizmen/DATA/REPOS/ai-apps/dev/safety-hat/datasets/dataset_v1/valid/labels.cache... 90 images, 0 backgrounds, 0 corrupt: 100%|██████████| 90/90 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: Adam(lr=0.001, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.000515625), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train3
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      1.21G        nan        nan        nan         64        640:  68%|██████▊   | 135/199 [00:49<00:23,  2.72it/s]


KeyboardInterrupt: 